In [27]:
semestri = []

for i in range(18,25):
    leto = "FRI " + "20" + str(i) + "/20" + str(i + 1)
    semestri.append(leto + ", zimski semester")
    semestri.append(leto + ", letni semester")
    
semestri.remove("FRI 2018/2019, letni semester") #na spletni strani ni podatka o skupinah
semestri


['FRI 2018/2019, zimski semester',
 'FRI 2019/2020, zimski semester',
 'FRI 2019/2020, letni semester',
 'FRI 2020/2021, zimski semester',
 'FRI 2020/2021, letni semester',
 'FRI 2021/2022, zimski semester',
 'FRI 2021/2022, letni semester',
 'FRI 2022/2023, zimski semester',
 'FRI 2022/2023, letni semester',
 'FRI 2023/2024, zimski semester',
 'FRI 2023/2024, letni semester',
 'FRI 2024/2025, zimski semester',
 'FRI 2024/2025, letni semester']

pripravim selenium

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


BASE_URL = "https://urnik.fri.uni-lj.si/timetable" 
all_data = []

driver = webdriver.Chrome()
driver.get(BASE_URL)


dobim dejanske semestre iz HTML

In [29]:

try:

    p_element = driver.find_element(By.TAG_NAME, "p")
    links = p_element.find_elements(By.TAG_NAME, "a")




except Exception as e:
    print(f"Napaka pri semestru: {e}")

seznam = [(link.text, link.get_attribute("href")) for link in links]
seznam.reverse()

prve_pojavitve = {}

for text, href in seznam:
    for sem in semestri:
        if sem in text and sem not in prve_pojavitve:
            prve_pojavitve[sem] = (text, href)


rezultat = list(prve_pojavitve.values())
rezultat


[('FRI 2024/2025, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski'),
 ('FRI 2023/2024, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-zimski'),
 ('FRI 2023/2024, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-letni'),
 ('FRI 2024/2025, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-letni'),
 ('FRI 2020/2021, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski'),
 ('FRI 2021/2022, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-zimski'),
 ('FRI 2022/2023, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-zimski'),
 ('FRI 2022/2023, letni semester-1-10',
  'https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-letni-1-10'),
 ('FRI 2021/2022, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-letni-1-1'),
 ('FRI 2020/2021, letni semester-1-9',
  'https://urnik.fri.uni-lj.si/timetable/fri-2

dobim podatke o predmetih po semestrih za letnike

In [30]:
predmeti = []

# povezave na semester
for semester, povezava in rezultat:
    driver.get(povezava)

    # 1. letnik
    try:
        link_element = driver.find_element(By.XPATH, "//a[contains(text(), '1. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')]")
        link_element.click()
        elements_with_p = [element for element in driver.find_elements(By.XPATH, "//a[contains(text(), '_P')]") if element.text.endswith('_P')]
        for element in elements_with_p:
            text = element.text
            href = element.get_attribute("href")
            predmeti.append((text, href, semester, 1))
    except Exception as e:
        print(f"[Napaka - VSS1] Semester: {semester}, URL: {povezava}\n{e}")

    # 2. letnik
    try:
        driver.get(povezava)
        link_element = driver.find_element(By.XPATH, "//a[contains(text(), '2. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')]")
        link_element.click()
        elements_with_p = [element for element in driver.find_elements(By.XPATH, "//a[contains(text(), '_P')]") if element.text.endswith('_P')]
        for element in elements_with_p:
            text = element.text
            href = element.get_attribute("href")
            predmeti.append((text, href, semester, 2))
    except Exception as e:
        print(f"[Napaka - VSS2] Semester: {semester}, URL: {povezava}\n{e}")

    # 3. letnik
    try:
        driver.get(povezava)
        link_element = driver.find_element(By.XPATH, "//a[contains(text(), '3. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')]")
        link_element.click()
        elements_with_p = [element for element in driver.find_elements(By.XPATH, "//a[contains(text(), '_P')]") if element.text.endswith('_P')]
        for element in elements_with_p:
            text = element.text
            href = element.get_attribute("href")
            predmeti.append((text, href, semester, 3))
    except Exception as e:
        print(f"[Napaka - VSS3] Semester: {semester}, URL: {povezava}\n{e}")


In [31]:
kratica_ime = {}

for kratica, povezava, semester, letnik in predmeti:
    driver.get(povezava)

    try:
        subtitle_element = driver.find_element(By.CLASS_NAME, "subtitle")
        kratica_ime[kratica] = subtitle_element.text
    except Exception as e:
        print(f"Napaka pri pridobivanju elementa z razredom 'subtitle' za {kratica}: {e}")



kratica_ime je slovar v katerem se nahaj kratica predmeta in ime predmeta (kratica: ime)

v seznamu predmeti se nahaja terka z (kratica, povezava, semester, letnik)

In [32]:
import pandas as pd


def pretvori_semester(semester):
    leto1 = semester[6:8]
    leto2 = semester[11:13]
    if "zimski" in semester:
        sem = 1
    elif "letni" in semester:
        sem = 2
    return f"{leto1}/{leto2}/{sem}"

# priprava podatkov za DataFrame
data = []
for kratica, _, semester, letnik in predmeti:
    ime = kratica_ime.get(kratica, "Neznano ime")
    nov_semester = pretvori_semester(semester)
    data.append((kratica, ime, nov_semester, letnik))

# ustvarimo DataFrame
df_predmeti = pd.DataFrame(data, columns=["kratica", "ime", "semester", "letnik"])


driver.quit()

df_predmeti.head(10)

,kratica,ime,semester,letnik
0,DS(63705)_P,Diskretne strukture,24/25/1,1
1,RA_P,Računalniška arhitektura,24/25/1,1
2,M_P,Matematika,24/25/1,1
3,P1_P,Programiranje 1,24/25/1,1
4,UR_P,Uvod v računalništvo,24/25/1,1
5,EMP_P,Elektronsko in mobilno poslovanje,24/25/1,2
6,KPOV_P,Komunikacijski protokoli in omrežna varnost,24/25/1,2
7,TV_P,Tehnične veščine,24/25/1,2
8,UI(63720)_P,Umetna inteligenca,24/25/1,2
9,_P,Tehnične veščine 2,24/25/1,2


od tu naprej pridobivam podatke o posameznih študentih

ustvarimo seznam vpisnih številk

In [33]:
vpisne_stevilke = [
    int(f"63{str(dv).zfill(2)}0{str(tri).zfill(3)}")
    for dv in range(18, 25)
    for tri in range(0, 501)
]

vpisne_stevilke[0:10]

[63180000,
 63180001,
 63180002,
 63180003,
 63180004,
 63180005,
 63180006,
 63180007,
 63180008,
 63180009]

ustvarim tabelo s predmeti za studente

In [34]:

from selenium.common.exceptions import TimeoutException


# --- Funkcije ---

def leto_vpisa_iz_vpisne(vpisna):
    return 2000 + int(str(vpisna)[2:4])  # npr. 63220299 -> 2022

def leto_iz_semesterskega_opisa(opis):
    try:
        leto1 = int(opis[6:8])  # "24" iz "FRI 2024/2025..."
        return 2000 + leto1
    except:
        return 0  # če gre kaj narobe


# --- Glavna koda ---

#vpisne_stevilke = [63220299, 63240123, 63230146] #zbriši to vrstico za uporabo vseh vpisnih (zna trajat!!!)
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

podatki = []

for vpisna in vpisne_stevilke:
    leto_vpisa = leto_vpisa_iz_vpisne(vpisna)

    for semester_opis, povezava in rezultat:
        leto_sem = leto_iz_semesterskega_opisa(semester_opis)

        if leto_sem < leto_vpisa:
            continue

        driver.get(povezava)

        try:
            input_element = wait.until(EC.presence_of_element_located((By.ID, "id_student")))
            input_element.clear()
            input_element.send_keys(str(vpisna))

            submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Show timetable']")
            submit_button.click()

            try:
                wait_short = WebDriverWait(driver, 1)
                wait_short.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(text(), '_P')]")))

                predmeti = [
                    el.text for el in driver.find_elements(By.XPATH, "//a[contains(text(), '_P')]")
                    if el.text.endswith('_P')
                ]

                for predmet in predmeti:
                    podatki.append({
                        "vpisna": vpisna,
                        "predmet": predmet,
                        "semester": pretvori_semester(semester_opis)
                    })

            except TimeoutException:
                continue

        except Exception as e:
            print(f"Napaka pri vpisni {vpisna}, semester {semester_opis}: {e}")
            continue

driver.quit()

# Ustvari DataFrame
df_studenti = pd.DataFrame(podatki)
df_studenti.head(10)


,vpisna,predmet,semester
0,63180001,IOI_P,24/25/1
1,63180001,NPMP_P,24/25/1
2,63180001,RSO_P,24/25/1
3,63180001,RS(63509)_P,24/25/2
4,63180001,VZR_P,24/25/2
5,63180001,SMRPO_P,24/25/2
6,63180001,RIS(63268)_P,20/21/2
7,63180001,RK_P,19/20/2
8,63180001,PPJ_P,19/20/2
9,63180001,APS2(63280)_P,19/20/2


Pretvorimo dataframe v csv

In [35]:
df_studenti.to_csv("podatki.csv",index=False)